In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [25]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label = self.data.iloc[idx, -1]
        features = torch.tensor(self.data.iloc[idx, :-1].values, dtype=torch.float32)
        return features, label

dataset = CustomDataset('pd_speech_features.csv')


In [26]:
import torchvision.models as models

model = models.resnet18(pretrained=False)
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 2)  # 2 classes: 0 or 1


d:\CODING\anaconda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\CODING\anaconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [29]:
from sklearn.model_selection import train_test_split

train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
import torch

def tenzor(input,c):
    # Assume your input has shape [batch_size, num_features]
    batch_size = input.size(0)
    if(c==0):
        num_features = input.size(1)
    else:
        num_features = 1
    # Reshape the input to [batch_size, 1, num_features, 1]
    input = torch.reshape(input, (batch_size, 1, num_features, 1))

    # Repeat the input along the channel dimension to create 3 channels
    input = input.repeat(1, 3, 1, 1)

    # Resize the input to 224x224 using interpolation
    input = torch.nn.functional.interpolate(input, size=(224, 224), mode='bilinear', align_corners=False)
    return input

val_loader = DataLoader(val_dataset, batch_size=32)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

num_epochs = 10

for epoch in range(num_epochs):
    train_loss = 0
    train_acc = 0
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        features=tenzor(features,0)
        outputs = model(features)
        # print(outputs,labels)
        # labels=labels.float()
        labels=labels.view(-1)
        # labels=tenzor(labels,0)
        # unique_targets = torch.unique(labels)
        # print(unique_targets)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * features.size(0)
        train_acc += (outputs.argmax(dim=1) == labels).sum().item()
    train_loss /= len(train_dataset)
    train_acc /= len(train_dataset)

    val_loss = 0
    val_acc = 0
    model.eval()
    with torch.no_grad():
        for features, labels in val_loader:
            features=tenzor(features,0)
            outputs = model(features)
            labels=labels.view(-1)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * features.size(0)
            val_acc += (outputs.argmax(dim=1) == labels).sum().item()
        val_loss /= len(val_dataset)
        val_acc /= len(val_dataset)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')


Epoch 1/10, Train Loss: 0.5479, Train Acc: 0.7632, Val Loss: 2.5901, Val Acc: 0.3289


KeyboardInterrupt: 

In [ ]:
test_dataset = CustomDataset('test_data.csv')
test_loader = DataLoader(test_dataset, batch_size=32)

model.eval()
test_loss = 0
test_acc = 0
with torch.no_grad():
    for features, labels in test_loader:
        outputs = model(features)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * features.size(0)
        test_acc += (outputs.argmax(dim=1) == labels).sum().item()
    test_loss /= len(test_dataset)
    test_acc /= len(test_dataset)

print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')


In [11]:
print(dataset.iloc[:,:-1].size(0))

AttributeError: 'list' object has no attribute 'iloc'